In [ ]:
# pip install textblob

In [ ]:
import os
from astropy.table import Table, vstack, unique #this package is similar to pandas but personally, I like it more for survey data
import pandas
import numpy as np
import pycountry
import matplotlib.pyplot as plt
from textblob import TextBlob

In [ ]:
##Read in your data. os.path.join makes it so that you can type in your path without slashes because windows hates slashes
##If you have an excel file or csv file, you can use that with pandas
##read_csv from pandas reads in the csv file
##Then we turn the data into an astropy Table

# #An example to deal with headers in the reading stage
# dat=pandas.read_excel(os.path.join('C:','\\','Users','elean','Documents','School','Python4STEM','SurveyData.xlsx'), header=0, skiprows=1)
# dataA=Table.from_pandas(dat)

#An example where you would want the first line to be your column names
dat=pandas.read_excel(os.path.join('C:','\\','Users','elean','Documents','School','Python4STEM','SurveyData.xlsx'))
dataA=Table.from_pandas(dat)

# #If you have a header but you don't want spaces in the questions (this is pretty unusual but qualtrics can be annoying with the double header)
for i in range(len(dataA.colnames)):
    dataA.rename_column(dataA.colnames[i],dataA.colnames[i].replace(" ",""))

data=pandas.read_csv(os.path.join('C:','\\','Users','elean','Documents','School','Python4STEM','SurveyData2.csv'))
dataB=Table.from_pandas(data)

for i in range(len(dataB.colnames)):
    dataB.rename_column(dataB.colnames[i],dataB.colnames[i].replace(" ",""))

In [ ]:
#Header problems
#We didn't want the data in row 0 (row after the header) so by doing [1:] we tell it to take the data after the 0th row
#But before that, we want to store the row 0 for reference because it has the actual questions

#By doing the data1[0:0], we created a new table with the same header and the same width as the data1 table
qRef=Table(dataA[0:0])
#This adds the row with the questions in them to the question reference table
qRef.add_row(dataB[0])



In [ ]:
data1=dataA[1:]
data2=dataB[1:]

In [ ]:
#Demonstrating stacking multiple datasets
#vstack stacks data vertically so you want to do this when your data have the same columns
result=vstack([data1,data2])
print(result)
result.to_pandas().to_csv(os.path.join('C:','\\','Users','elean','Documents','School',
                                                       'SurveyDataAll.csv'))

#oh no, the data types are different what do we do???

In [ ]:
#First we might try printing the data types of the two tables to visually see the difference
print(data1.dtype)
print(data2.dtype)

In [ ]:
#Wow there are a lot of differences
#We are just going to change the data types of all columns
#Here we are changing everything to a 256 bit string
data1=Table(np.array(data1),dtype=['<U256']*len(data1.colnames))
data2=Table(np.array(data2),dtype=['<U256']*len(data2.colnames))


In [ ]:
#Check if the data types of the columns you want to stack are the same
#Data in columns have to be the same to stack them

for colname in data1.colnames:
    if colname not in data2.colnames:
        print(colname, 'Not in data2')
        break
    if data1.dtype[colname] != data2.dtype[colname]:
        print(data1.dtype[colname], data2.dtype[colname], 'Not the same dtype')
        break

In [ ]:
#Demonstrating stacking multiple datasets
#vstack stacks data vertically so you want to do this when your data have the same columns
vstackR=vstack([data1,data2])

vstackR.to_pandas().to_csv(os.path.join('C:','\\','Users','elean','Documents','School','Python4STEM',
                                                       'SurveyDataAll.csv'))

In [ ]:
#I forgot what the questions were for the next section, so let's check! Btw this only looks this nice in Jupyter
qRef

In [ ]:
#Demonstrating selecting specific parts of a dataset by known item
#We want only the people who like cookies n' cream, vanilla, chocolate, and other
#But we might have a few problems like the n' and capitalization
#We must use "" instead of '' to deal with the n' and .lower() to deal with the capitalization
bestFlavors=Table(vstackR[0:0])
for i in range(len(vstackR)):
    if vstackR[i]['Q2.4'].lower() in ["cookies n' cream", "vanilla", "chocolate", "other"]:
        bestFlavors.add_row(vstackR[i])
    else:
        continue

In [ ]:
#We want to know what the flavors are for "other" and luckily they added them as text
#We can add those flavors that aren't nan to the Q2.4 column so we don't have to keep checking what "other" is
for i in range(len(bestFlavors)):
    if bestFlavors[i]['Q2.4'].lower()=='other':
        if bestFlavors[i]['Q2.4_TEXT']!='nan':
            bestFlavors[i]['Q2.4']=bestFlavors[i]['Q2.4_TEXT']
        else:
            continue
    else:
        continue
print(bestFlavors['Q2.4'])   

In [ ]:
#People may have answered the survey more than once. Let's only take the unique response IDs

uniqueResponse=unique(bestFlavors, keys=['ResponseId'])
print(uniqueResponse)

In [ ]:
#Demonstrating fixing countries
#Pretty annoying that people typed in their countries in all different formats. Let's fix that

for i in range(len(uniqueResponse)):
    try:
        country=pycountry.countries.lookup(uniqueResponse[i]['Q2.7'])
        uniqueResponse[i]['Q2.7']=country.name
        print(uniqueResponse[i]['Q2.7'])
    except:
        print('Did not recognize ', uniqueResponse[i]['Q2.7'])
        uniqueResponse[i]['Q2.7']=input('Please input the country name: ') #This will always be a string
    


In [ ]:
###Plotting data
###This section makes the plots larger to make it easier to see

urPandas=uniqueResponse.to_pandas()

urPandas[['Duration(inseconds)','Q2.5']]=urPandas[['Duration(inseconds)','Q2.5']].apply(pandas.to_numeric)


plt.scatter(urPandas['Duration(inseconds)'],urPandas['Q2.5'])
plt.xlabel("Duration in Seconds")
plt.ylabel("Age")

In [ ]:
#Sentiment analysis has a bunch of different uses and packages the most common being #nltk
#https://www.datacamp.com/community/tutorials/simplifying-sentiment-analysis-python
# https://www.datacamp.com/community/tutorials/text-analytics-beginners-nltk
# Lexicon-based: count number of positive and negative words in given text and the larger count will be the sentiment of text.
# Machine learning based approach: Develop a classification model, which is trained using the pre-labeled dataset of positive, negative, and neutral.

#Creates an empty sentiment column
# urPandas['sentiment']=""

# for i in urPandas['Q2.3']:
for i, row in urPandas.iterrows():
    blob=TextBlob(urPandas['Q2.3'][i])
    sentiment=blob.sentiment.polarity
    print(sentiment)
    urPandas.loc[i,'sentiment']=sentiment
print(urPandas)